In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110560

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,67806.0310
2018-02-28,59072.6276
2018-03-31,62446.0326
2018-04-30,71982.6945
2018-05-31,81552.0175
2018-06-30,87610.9639
2018-07-31,78444.4680
2018-08-31,99277.7328
2018-09-30,97655.4294


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31     67806.0310
 2018-02-28     59072.6276
 2018-03-31     62446.0326
 2018-04-30     71982.6945
 2018-05-31     81552.0175
 2018-06-30     87610.9639
 2018-07-31     78444.4680
 2018-08-31     99277.7328
 2018-09-30     97655.4294
 2018-10-31     88180.4949
 2018-11-30     86387.4910
 2018-12-31     90382.7928
 2019-01-31     91758.0623
 2019-02-28    107223.7531
 2019-03-31     95583.6411
 2019-04-30     93502.0372
 2019-05-31    105363.1373
 2019-06-30    120903.5294
 2019-07-31    118739.0423
 2019-08-31    110673.9068
 2019-09-30    102988.1358
 2019-10-31    106144.2288
 2019-11-30    110392.3878
 2019-12-31     97438.5509
 2020-01-31    101301.6302
 2020-02-29     93058.5224
 2020-03-31     73508.0003
 2020-04-30     81767.8791
 2020-05-31     87628.6068
 2020-06-30     96773.0657
 2020-07-31    113882.2571
 2020-08-31    104325.8057
 2020-09-30     81436.4804
 2020-10-31     88386.2246
 2020-11-30     79971.7091
 2020-12-31     78998.8764

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.440890
p-value : 0.130527
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -6.461940
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=933.923, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=911.902, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=908.637, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=907.781, Time=0.07 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=906.727, Time=0.06 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=908.018, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=897.906, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=899.322, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=900.980, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=902.732, Time=0.07 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=904.732, Time=0.08 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=899.850, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=901.039, Time=0.12 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=902.717, Time=0.10 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=904.688, Time=0.31 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -445.903
Method:                       css-mle   S.D. of innovations           9766.401
Date:                Sat, 11 Sep 2021   AIC                            897.805
Time:                        04:05:25   BIC                            903.018
Sample:                             0   HQIC                           899.716
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       8.757e+04   6357.378     13.775      0.000    7.51e+04       1e+05
ar.L1.y        0.7784      0.095      8.228      0.000       0.593       0.964
                                    Roots           

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([85620.87658946, 86052.75011345]), array([ 9766.40140314, 12376.4762666 ]), array([[ 66479.08158074, 104762.67159818],
       [ 61795.30237539, 110310.19785151]]))
